In [52]:
import numpy as np

#Nagel & Schreckenberg traffic simulation addapted from class that I'll use for each lane

class Lane:

    def __init__(self, road_length=100, car_density=0.2, prob_slow=0.5,
                 max_speed=5):
        '''
        Create a new traffic simulation object. Cars are distributed randomly
        along the road and start with random velocities.

        Inputs:

            road_length (int) The number of cells in the road. Default: 100.

            car_density (float) The fraction of cells that have a car on them.
              Default: 0.2.

            prob_slow (float) The probability that a car will randomly slow down
              by 1 during an update step. Default: 0.5.

            max_speed (int) The maximum speed in car cells per update step.
              Default: 5.
        '''
        self.road_length = road_length
        self.car_density = car_density
        self.max_speed = max_speed
        self.prob_slow = prob_slow

        # Create an empty road: -1 means empty in this simulation
        self.state = np.full(self.road_length+1, -1, dtype=int)
        # Choose random locations to place cars
        random_indexes = np.random.choice(
            range(self.road_length),
            size=int(round(car_density * self.road_length)),
            replace=False)
        # Give each car a random initial speed from 0 to max_speed
        self.state[random_indexes] = np.random.randint(
            0, self.max_speed + 1, size=len(random_indexes))

        self.state[-1]=0

        # Keep track of the time steps and average traffic flow at each step
        self.time_step = 0
        self.average_traffic_flow = []

    def update(self):
        '''
        Advance one time step in the simulation.
        '''

        # Update car speeds
        for i in range(self.road_length):
            if self.state[i] != -1:
                # Distance is the number of empty cells between this car and the
                # next car plus 1
                distance = 1
                while self.state[(i + distance)] == -1:
                    distance += 1
                # Acceleration
                if self.state[i] < self.max_speed:
                    self.state[i] += 1
                # Deceleration
                if self.state[i] >= distance:
                    self.state[i] = distance - 1
                # Randomization
                if (
                    (self.state[i] > 0) and
                    (np.random.uniform() < self.prob_slow)
                ):
                    self.state[i] -= 1

        # Move cars forward using their new speeds
        new_state = np.full(self.road_length+1, -1, dtype=int)
        new_state[-1] = 0
        for i in range(self.road_length):
            cell = self.state[i]
            if cell != -1:
                new_state[i + cell] = cell
        self.state = new_state

        # Update average traffic flow history
        self.average_traffic_flow.append(
            sum(self.state[self.state > 0]) / self.road_length)
        self.time_step += 1

    def display(self):
        '''
        Print out the current state of the simulation.
        '''
        print(''.join('.' if x == -1 else str(x) for x in self.state[:-1]))


sim = Lane(
    road_length=50, car_density=0.2, max_speed=5, prob_slow=0.5)
sim.state = sim.state.copy()
for i in range(20):
    sim.display()
    sim.update()

.......3........2..2..3..3....3..........1.0.3..1.
...........4.....1..1..1.....4....4......0.0..1.0.
................5.1...2.1........4.....5..10..0.0.
................0...2..1.1............5..20.1.0..1
................0....1..1..2............200.0..1.0
................0......2.1...2..........00.1.1.0.0
................0......0...2....3.......00..1.1.10
.................1.....0......3.....4...0.1.0.0.00
...................2...0..........4....3.1.1.1.100
.....................2.0.............3.0..1.10.000
.....................0.0..............1.1.0.0.1000
.....................0..1.............0.0..1.10000
......................1...2............1.1.0.00000
.......................1.....3.........0..1.100000
.........................2......3.......1..1000000
...........................2........4....1.0000000
.............................2.........3.0.0000000
................................3......0..10000000
....................................4..0..00000000
...............................

In [85]:
#class for connecting multiple lanes 

class Node:
    
    def __init__(self, lanes_from, lanes_to):
        self.lanes_from = lanes_from
        self.lanes_to = lanes_to

    def removeFromQueue(self, dirFrom):

        arr = np.array([i for i in self.lanes_to.keys()])
        boolArr = arr != dirFrom
        newArr = arr[boolArr]
        newArr = arr[arr != dirFrom]

        dirTo = np.random.choice(newArr)

        if self.lanes_to[dirTo].state[0] == -1:
            self.lanes_from[dirFrom].state[-2] = -1
            self.lanes_to[dirTo].state[0] = 0
            

    def Move_to_new_lane(self):
        for dir,lane in self.lanes_from.items():
            if lane.state[-2] > -1:
                self.removeFromQueue(dir)
        

In [86]:
#class for running everything

class System:
    
    def __init__(self, nodes, lanes):
        self.nodes = nodes
        self.lanes = lanes


In [87]:
lanes = {'down':Lane(road_length=20),
         'up':Lane(road_length=20)}

nodes = [Node(lanes,lanes)]

for i in range(20):
    for dir,lane in lanes.items():
        print(i)
        lane.update()
        print(dir)
        lane.display()
        print('\n')
    for node in nodes:
        node.Move_to_new_lane()


0
down
......30.....4...3..


0
up
....3...........00.0


1
down
.1....0.1......2..1.


1
up
........4.......0.1.


2
down
..1...0...2.....1..1


2
up
.............5..0..1


3
down
0..1...1.....3...1..


3
up
0.............1.0...


4
down
0....2...2.....2..1.


4
up
.1............0..1..


5
down
0.......3..2....1..1


5
up
..1...........0...1.


6
down
.1........2...3...2.


6
up
0..1...........1..0.


7
down
...2........2....30.


7
up
.1...2...........20.


8
down
......3........3.00.


8
up
..1.....3........0.1


9
down
.1.......3.....0.00.


9
up
....2......3......1.


10
down
...2.........4..100.


10
up
......2.......3...0.


11
down
.....2........1.000.


11
up
........2.......2.0.


12
down
.......2......0.00.1


12
up
...........3....0..1


13
down
.1........3....100..


13
up
0..............40...


14
down
..1...........400.1.


14
up
0..............00...


15
down
...1..........000..1


15
up
0..............0.1..


16
down
....1.........000..0


16
up
.1..............1.1.




In [88]:
#test with more roads

lanes = {'1d':Lane(road_length=20),
         '1u':Lane(road_length=20),
         '2r':Lane(road_length=20),
         '2l':Lane(road_length=20),
         '3d':Lane(road_length=20),
         '3u':Lane(road_length=20),
         '4r':Lane(road_length=20),
         '4l':Lane(road_length=20)}

nodes = [Node({'down':lanes['1u'],'right':lanes['2l']},{'down':lanes['1d'],'right':lanes['2r']}),
         Node({'down':lanes['3u'],'left':lanes['2r']},{'down':lanes['3d'],'left':lanes['2l']}),
         Node({'up':lanes['1d'],'right':lanes['4l']},{'up':lanes['1u'],'right':lanes['4r']}),
         Node({'up':lanes['3d'],'left':lanes['4r']},{'up':lanes['3u'],'left':lanes['4l']})]

for i in range(10):
    for dir,lane in lanes.items():
        print(i)
        lane.update()
        print(dir)
        lane.display()
        print('\n')
    for node in nodes:
        node.Move_to_new_lane()

#I need a better way of printing this
#but it looks like its working
#the number of cars in the beggining is the same as in the end :)

0
1d
.0..2..2.....4......


0
1u
....3........1.10...


0
2r
..........20..1....3


0
2l
00..2......2........


0
3d
..........2...200...


0
3u
...2........5...1..2


0
4r
....2......1.....50.


0
4l
..10.1...........0..


1
1d
..1...2..2........5.


1
1u
.......3......10.1..


1
2r
..........0.1..1....


1
2l
0.1....3.....2......


1
3d
.1...........300.1..


1
3u
......3........3.1.0


1
4r
......2.....1....0.1


1
4l
..00..1...........1.


2
1d
...1...1...2......0.


2
1u
...........4..00..1.


2
2r
..........0..1..1...


2
2l
0..1......3....2....


2
3d
...2.........00.1.1.


2
3u
.1.......3.....0..10


2
4r
........2....1....1.


2
4l
..00....2.........0.


3
1d
.....2...2...2....0.


3
1u
.............20.1.0.


3
2r
..........0....2..2.


3
2l
.1...2........4..2..


3
3d
......3......00.0..1


3
3u
...2.........4..1.00


3
4r
...........3...2..0.


3
4l
..00......2.......0.


4
1d
........3...3..2..0.


4
1u
.............00.0.0.


4
2r
..........0.....1..1


4
2l
0.1....2.......

In [93]:
#insert images of the streets
lanes = {'Farr_1_d':Lane(road_length=10),
         'Farr_1_u':Lane(road_length=10, car_density=0.3),
         'Farr_2_d':Lane(road_length=20),
         'Farr_2_u':Lane(road_length=20),
         'Farr_3_d':Lane(road_length=10),
         'Farr_3_u':Lane(road_length=10),

         'Cler_1_l':Lane(road_length=10, car_density=0.3),
         'Cler_1_r':Lane(road_length=10, car_density=0.3),
         'Cler_2_l':Lane(road_length=30, car_density=0.3),
         'Cler_2_r':Lane(road_length=30, car_density=0.3),
         
         'Rose_1_d':Lane(road_length=25, car_density=0.3),
         'Rose_1_u':Lane(road_length=25),
         'Rose_2_d':Lane(road_length=15),
         'Rose_2_u':Lane(road_length=15),
         'Rose_3_d':Lane(road_length=10),
         'Rose_3_u':Lane(road_length=10),
         
         'Warn_l':Lane(road_length=20),
         'Warn_r':Lane(road_length=20, car_density=0.3),

         'Rays_l':Lane(road_length=10),
         'Rays_r':Lane(road_length=10),
         
         'Toph_l':Lane(road_length=10, car_density=0.4),
         'Toph_r':Lane(road_length=10, car_density=0.4),
         
         'Bake_u':Lane(road_length=20),
         'Bake_d':Lane(road_length=20),
         
         'Herb_u':Lane(road_length=15),
         'Herb_d':Lane(road_length=15),}

nodes = [Node({'down':lanes['Rose_3_u'],
                'right':lanes['Farr_1_u']},
              {'down':lanes['Rose_3_d'],
                'right':lanes['Farr_1_d']}),

         Node({'down':lanes['Rose_2_u'],
                'right':lanes['Toph_l'],
                'up':lanes['Rose_3_d']},
              {'down':lanes['Rose_2_d'],
                'right':lanes['Toph_r'],
                'up':lanes['Rose_3_u']}),

         Node({'down':lanes['Bake_u'],
                'right':lanes['Farr_2_u'],
                'left':lanes['Toph_r'],
                'up':lanes['Farr_1_d']},
              {'down':lanes['Bake_d'],
                'right':lanes['Farr_2_d'],
                'left':lanes['Toph_l'],
                'up':lanes['Farr_1_u']}),

         Node({'down':lanes['Rose_1_u'],
                'right':lanes['Warn_l'],
                'up':lanes['Rose_2_d']},
              {'down':lanes['Rose_1_d'],
                'right':lanes['Warn_r'],
                'up':lanes['Rose_2_u']}),

         Node({'down':lanes['Herb_u'],
                'right':lanes['Rays_l'],
                'left':lanes['Warn_r'],
                'up':lanes['Bake_d']},
              {'down':lanes['Herb_d'],
                'right':lanes['Rays_r'],
                'left':lanes['Warn_l'],
                'up':lanes['Bake_u']}),

         Node({'down':lanes['Farr_3_u'],
                'left':lanes['Rays_l'],
                'up':lanes['Farr_2_d']},
              {'down':lanes['Farr_3_d'],
                'left':lanes['Rays_r'],
                'up':lanes['Farr_2_u']}),

         Node({'up':lanes['Rose_1_d'],
                'right':lanes['Cler_2_l']},
              {'up':lanes['Rose_1_u'],
                'right':lanes['Cler_2_r']}),

         Node({'right':lanes['Cler_1_l'],
                'left':lanes['Cler_2_r'],
                'up':lanes['Herb_d']},
              {'right':lanes['Cler_1_r'],
                'left':lanes['Cler_2_l'],
                'up':lanes['Herb_u']}),

         Node({'up':lanes['Farr_3_d'],
                'left':lanes['Cler_1_l']},
              {'up':lanes['Farr_3_u'],
                'left':lanes['Cler_1_r']}),]



In [95]:
for i in range(20):
    for dir,lane in lanes.items():
        print(i)
        lane.update()
        print(dir)
        lane.display()
        print('\n')
    for node in nodes:
        node.Move_to_new_lane()

0
Farr_1_d
0.....2...


0
Farr_1_u
.1.......0


0
Farr_2_d
...2...2........1.1.


0
Farr_2_u
0......3.........50.


0
Farr_3_d
.1....3...


0
Farr_3_u
........10


0
Cler_1_l
........0.


0
Cler_1_r
..1...1000


0
Cler_2_l
.........2.1.......2000..1....


0
Cler_2_r
0.1...2.....3..0.1.1..1..2..1.


0
Rose_1_d
..1..2........1...1.1..1.


0
Rose_1_u
.....1..............10.0.


0
Rose_2_d
..1.......300.0


0
Rose_2_u
...1.......2...


0
Rose_3_d
.1....2...


0
Rose_3_u
.1..2...2.


0
Warn_l
............2.1..10.


0
Warn_r
00................20


0
Rays_l
..........


0
Rays_r
0.00..1.00


0
Toph_l
....10....


0
Toph_r
.0.00000..


0
Bake_u
0........2.....1....


0
Bake_d
.1.....2.......2.1.1


0
Herb_u
.1........0.00.


0
Herb_d
........10.000.


1
Farr_1_d
.1.......3


1
Farr_1_u
..1.......


1
Farr_2_d
......3...3......1.1


1
Farr_2_u
.1.........4.....0.1


1
Farr_3_d
..1.....2.


1
Farr_3_u
........00


1
Cler_1_l
........0.


1
Cler_1_r
....2.0000


1
Cler_2_l
..........1..2.....000.